In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from tabulate import tabulate
df=pd.read_csv(r"/content/drive/MyDrive/Acorn/Project/공공자전거_대여소_정보(22.12월 기준).csv")
df_rental=pd.read_csv(r"/content/drive/MyDrive/Acorn/Project/2020data/강남대여소_2020.csv")
df_weather=pd.read_csv(r"/content/drive/MyDrive/Acorn/Project/2020data/2020_weather_processed.csv")
df_air=pd.read_csv(r"/content/drive/MyDrive/Acorn/Project/2020data/2020_air_processed.csv")
df_people=pd.read_csv(r"/content/drive/MyDrive/Acorn/Project/2020data/2020_people_processed.csv")

In [3]:
print('대여소 정보 22 12월 기준')
print(tabulate(df.head(3), headers='keys', tablefmt='psql', showindex=True))
print()
print('강남대여소 2020')
print(tabulate(df_rental.head(3), headers='keys', tablefmt='psql', showindex=True))
print()
print('강남대여소 공기오염도 2020')
print(tabulate(df_air.head(3), headers='keys', tablefmt='psql', showindex=True))
print()
print('유동인구 2020')
print(tabulate(df_people.head(3), headers='keys', tablefmt='psql', showindex=True))
print()

대여소 정보 22 12월 기준
+----+--------------+---------------------+----------+------------------------------------------------------+---------+---------+------------+-----------------+----------------+------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|    |   대여소번호 | 보관소(대여소)명    | 자치구   | 상세주소                                             |    위도 |    경도 | 설치시기   |   거치대수(LCD) |   거치대수(QR) | 운영방식   |   Unnamed: 10 |   Unnamed: 11 |   Unnamed: 12 |   Unnamed: 13 |   Unnamed: 14 |   Unnamed: 15 |   Unnamed: 16 |   Unnamed: 17 |   Unnamed: 18 |
|----+--------------+---------------------+----------+------------------------------------------------------+---------+---------+------------+-----------------+----------------+------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------|
|  0 |          3

# 20-21 강남대여소에 대여소코드 넣기

In [4]:
print(tabulate(df.head(3), headers='keys', tablefmt='psql', showindex=True))
print()
print(df.columns)
df=df.drop(columns=['자치구', '상세주소', '위도', '경도', '설치시기', '거치대수(LCD)',
       '거치대수(QR)', '운영방식', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18'])

print(tabulate(df.head(3), headers='keys', tablefmt='psql', showindex=True))

+----+--------------+---------------------+----------+------------------------------------------------------+---------+---------+------------+-----------------+----------------+------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|    |   대여소번호 | 보관소(대여소)명    | 자치구   | 상세주소                                             |    위도 |    경도 | 설치시기   |   거치대수(LCD) |   거치대수(QR) | 운영방식   |   Unnamed: 10 |   Unnamed: 11 |   Unnamed: 12 |   Unnamed: 13 |   Unnamed: 14 |   Unnamed: 15 |   Unnamed: 16 |   Unnamed: 17 |   Unnamed: 18 |
|----+--------------+---------------------+----------+------------------------------------------------------+---------+---------+------------+-----------------+----------------+------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------|
|  0 |          301 | 경복궁역 7번출구 앞 

In [5]:
df = df.rename(columns={'보관소(대여소)명': '대여 대여소명'})

print(tabulate(df.head(3), headers='keys', tablefmt='psql', showindex=True))

+----+--------------+---------------------+
|    |   대여소번호 | 대여 대여소명       |
|----+--------------+---------------------|
|  0 |          301 | 경복궁역 7번출구 앞 |
|  1 |          302 | 경복궁역 4번출구 뒤 |
|  2 |          303 | 광화문역 1번출구 앞 |
+----+--------------+---------------------+



# 대여소+강수량 데이터

In [6]:
# df 데이터프레임의 '대여 대여소명' 열의 앞 뒤 공백 제거
df['대여 대여소명'] = df['대여 대여소명'].str.strip()
df_rental['대여 대여소명'] = df_rental['대여 대여소명'].str.strip()

# df_rental과 df 테이블을 대여 대여소명을 기준으로 조인
df_rental = pd.merge(df_rental, df, on='대여 대여소명', how='left')

print(tabulate(df_rental.head(50), headers='keys', tablefmt='psql', showindex=True))

+----+------------------------+------------+----------+------------+--------------+
|    | 대여 대여소명          | 날짜       |   시간대 |   이용건수 |   대여소번호 |
|----+------------------------+------------+----------+------------+--------------|
|  0 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        0 |          1 |         2335 |
|  1 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        1 |          0 |         2335 |
|  2 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        2 |          0 |         2335 |
|  3 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        3 |          0 |         2335 |
|  4 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        4 |          0 |         2335 |
|  5 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        5 |          0 |         2335 |
|  6 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        6 |          0 |         2335 |
|  7 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        7 |          1 |         2335 |
|  8 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        8 |          0 |         2335 |
|  9 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        9 |          0 |         2335 |
| 10 | 3호선 매봉역 3번출

In [7]:
print(tabulate(df_rental.tail(50), headers='keys', tablefmt='psql', showindex=True))

+--------+-----------------+------------+----------+------------+--------------+
|        | 대여 대여소명   | 날짜       |   시간대 |   이용건수 |   대여소번호 |
|--------+-----------------+------------+----------+------------+--------------|
| 836123 | 크린랩 본사     | 2020-12-29 |       22 |          0 |         2386 |
| 836124 | 크린랩 본사     | 2020-12-29 |       23 |          0 |         2386 |
| 836125 | 크린랩 본사     | 2020-12-30 |        0 |          0 |         2386 |
| 836126 | 크린랩 본사     | 2020-12-30 |        1 |          0 |         2386 |
| 836127 | 크린랩 본사     | 2020-12-30 |        2 |          0 |         2386 |
| 836128 | 크린랩 본사     | 2020-12-30 |        3 |          0 |         2386 |
| 836129 | 크린랩 본사     | 2020-12-30 |        4 |          0 |         2386 |
| 836130 | 크린랩 본사     | 2020-12-30 |        5 |          0 |         2386 |
| 836131 | 크린랩 본사     | 2020-12-30 |        6 |          0 |         2386 |
| 836132 | 크린랩 본사     | 2020-12-30 |        7 |          0 |         2386 |
| 836133 | 크린랩 본사

In [8]:
print(df_weather.columns)
print()

# '일시' 열의 이름을 '날짜'로 변경
df_weather = df_weather.rename(columns={'일시': '날짜'})

# 결과 출력
print(tabulate(df_weather.head(3), headers='keys', tablefmt='psql', showindex=True))

Index(['지점', '지점명', '일시', '평균기온(°C)', '일강수량(mm)'], dtype='object')

+----+--------+----------+------------+----------------+----------------+
|    |   지점 | 지점명   | 날짜       |   평균기온(°C) | 일강수량(mm)   |
|----+--------+----------+------------+----------------+----------------|
|  0 |    108 | 서울     | 2020-01-01 |           -2.2 | 비옴           |
|  1 |    108 | 서울     | 2020-01-02 |            1   | 비안옴         |
|  2 |    108 | 서울     | 2020-01-03 |           -0.1 | 비안옴         |
+----+--------+----------+------------+----------------+----------------+


In [9]:
# 날짜를 기준으로 두 데이터프레임 병합
merged_df = pd.merge(df_rental, df_weather, on='날짜', how='left')

# 필요한 경우 칼럼명을 조정하여 원하는 형식으로 만들기
merged_df.rename(columns={'일강수량(mm)': '날씨'}, inplace=True)

# 결과 출력
print(merged_df.columns)
print()

print(tabulate(merged_df.head(3), headers='keys', tablefmt='psql', showindex=True))
print()

# 합친 데이터프레임에서 필요한 칼럼만 가져오기
selected_columns = ['대여소번호','대여 대여소명', '날짜', '시간대', '날씨',  '평균기온(°C)']
subset_df = merged_df[selected_columns]
print(tabulate(subset_df.head(3), headers='keys', tablefmt='psql', showindex=True))
print(subset_df.shape)

Index(['대여 대여소명', '날짜', '시간대', '이용건수', '대여소번호', '지점', '지점명', '평균기온(°C)', '날씨'], dtype='object')

+----+------------------------+------------+----------+------------+--------------+--------+----------+----------------+--------+
|    | 대여 대여소명          | 날짜       |   시간대 |   이용건수 |   대여소번호 |   지점 | 지점명   |   평균기온(°C) | 날씨   |
|----+------------------------+------------+----------+------------+--------------+--------+----------+----------------+--------|
|  0 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        0 |          1 |         2335 |    108 | 서울     |           -2.2 | 비옴   |
|  1 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        1 |          0 |         2335 |    108 | 서울     |           -2.2 | 비옴   |
|  2 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        2 |          0 |         2335 |    108 | 서울     |           -2.2 | 비옴   |
+----+------------------------+------------+----------+------------+--------------+--------+----------+----------------+--------+

+----+--------------+------------------------+------------+---

# 대여소+강수량+(공기오염도 - 평균값을 넣기)

In [10]:
print(df_air.columns)
print(tabulate(df_air.head(3), headers='keys', tablefmt='psql', showindex=True))

Index(['측정일시', '측정소 코드', '측정항목', '평균값', '측정일', '측정시간'], dtype='object')
+----+-------------+---------------+------------+----------+-------------+------------+
|    |    측정일시 |   측정소 코드 |   측정항목 |   평균값 |      측정일 |   측정시간 |
|----+-------------+---------------+------------+----------+-------------+------------|
|  0 | 2.02001e+09 |           123 |          9 |       24 | 2.02001e+07 |        nan |
|  1 | 2.02001e+09 |           123 |          9 |       21 | 2.02001e+07 |          1 |
|  2 | 2.02001e+09 |           123 |          9 |       15 | 2.02001e+07 |          2 |
+----+-------------+---------------+------------+----------+-------------+------------+


In [11]:
# '일시' 열의 이름을 '날짜'로 변경
df_air = df_air.rename(columns={'측정일': '날짜'})

# 결과 출력
print(tabulate(df_air.head(3), headers='keys', tablefmt='psql', showindex=True))

+----+-------------+---------------+------------+----------+-------------+------------+
|    |    측정일시 |   측정소 코드 |   측정항목 |   평균값 |        날짜 |   측정시간 |
|----+-------------+---------------+------------+----------+-------------+------------|
|  0 | 2.02001e+09 |           123 |          9 |       24 | 2.02001e+07 |        nan |
|  1 | 2.02001e+09 |           123 |          9 |       21 | 2.02001e+07 |          1 |
|  2 | 2.02001e+09 |           123 |          9 |       15 | 2.02001e+07 |          2 |
+----+-------------+---------------+------------+----------+-------------+------------+


In [12]:
# 날짜 형식 변경
df_air['날짜'] = pd.to_datetime(df_air['날짜'], format='%Y%m%d')

# 날짜를 원하는 형식으로 다시 문자열로 변환
df_air['날짜'] = df_air['날짜'].dt.strftime('%Y-%m-%d')

#측정시간 NaN값 0으로
df_air['측정시간'] = df_air['측정시간'].fillna(0)
print(tabulate(df_air.head(3), headers='keys', tablefmt='psql', showindex=True))

+----+------------+---------------+------------+----------+------------+------------+
|    |   측정일시 |   측정소 코드 |   측정항목 |   평균값 | 날짜       |   측정시간 |
|----+------------+---------------+------------+----------+------------+------------|
|  0 | 2020010100 |           123 |          9 |       24 | 2020-01-01 |          0 |
|  1 | 2020010101 |           123 |          9 |       21 | 2020-01-01 |          1 |
|  2 | 2020010102 |           123 |          9 |       15 | 2020-01-01 |          2 |
+----+------------+---------------+------------+----------+------------+------------+


In [13]:
# df_air 칼럼의 측정시간 값과 일치하는 것을 기준으로 두 데이터프레임을 날짜를 기준으로 병합
merged_df = pd.merge(subset_df, df_air, left_on=['날짜', '시간대'], right_on=['날짜', '측정시간'], how='left')

# 필요한 경우 칼럼명을 조정하여 원하는 형식으로 만들기
merged_df.rename(columns={'평균값': 'Pm2.5'}, inplace=True)

# 결과 출력
print(tabulate(merged_df.head(3), headers='keys', tablefmt='psql', showindex=True))

+----+--------------+------------------------+------------+----------+--------+----------------+------------+---------------+------------+---------+------------+
|    |   대여소번호 | 대여 대여소명          | 날짜       |   시간대 | 날씨   |   평균기온(°C) |   측정일시 |   측정소 코드 |   측정항목 |   Pm2.5 |   측정시간 |
|----+--------------+------------------------+------------+----------+--------+----------------+------------+---------------+------------+---------+------------|
|  0 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        0 | 비옴   |           -2.2 | 2020010100 |           123 |          9 |      24 |          0 |
|  1 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        1 | 비옴   |           -2.2 | 2020010101 |           123 |          9 |      21 |          1 |
|  2 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        2 | 비옴   |           -2.2 | 2020010102 |           123 |          9 |      15 |          2 |
+----+--------------+------------------------+------------+----------+--------+----------------+----

In [14]:
# 제외할 열 목록
excluded_columns = ['측정일시', '측정소 코드', '측정항목', '측정시간']

# 제외한 열만 선택하여 새로운 데이터프레임 생성
final_df = merged_df.drop(columns=excluded_columns)
print(tabulate(final_df.head(3), headers='keys', tablefmt='psql', showindex=True))

+----+--------------+------------------------+------------+----------+--------+----------------+---------+
|    |   대여소번호 | 대여 대여소명          | 날짜       |   시간대 | 날씨   |   평균기온(°C) |   Pm2.5 |
|----+--------------+------------------------+------------+----------+--------+----------------+---------|
|  0 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        0 | 비옴   |           -2.2 |      24 |
|  1 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        1 | 비옴   |           -2.2 |      21 |
|  2 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        2 | 비옴   |           -2.2 |      15 |
+----+--------------+------------------------+------------+----------+--------+----------------+---------+


# 유동인구 추가하기

In [15]:
print(df_people.columns)
print(tabulate(df_people.head(3), headers='keys', tablefmt='psql', showindex=True))

Index(['기준일ID', '시간대구분', '총생활인구수', '대여소번호', '보관소(대여소)명'], dtype='object')
+----+------------+--------------+----------------+--------------+-----------------------------------+
|    |   기준일ID |   시간대구분 |   총생활인구수 |   대여소번호 | 보관소(대여소)명                  |
|----+------------+--------------+----------------+--------------+-----------------------------------|
|  0 |   20200101 |            0 |        8313.59 |         2358 | 구룡초사거리 (현대아파트10동 앞 ) |
|  1 |   20200101 |            0 |       25512.9  |         2325 | 대치삼성아파트 앞 교차로          |
|  2 |   20200101 |            0 |       46405.6  |         2343 | 세곡사거리 대왕파출소 앞          |
+----+------------+--------------+----------------+--------------+-----------------------------------+


In [16]:
# '보관소(대여소)명' 열의 이름을 '대여 대여소명'로 변경
df_people = df_people.rename(columns={'보관소(대여소)명': '대여 대여소명','기준일ID':'날짜','시간대구분':'시간대','총생활인구수':'유동인구(명)'})

# '유동인구(명)' 칼럼의 데이터를 int 타입으로 변환
df_people['유동인구(명)'] = df_people['유동인구(명)'].astype(int)

print(tabulate(df_people.head(3), headers='keys', tablefmt='psql', showindex=True))

+----+----------+----------+----------------+--------------+-----------------------------------+
|    |     날짜 |   시간대 |   유동인구(명) |   대여소번호 | 대여 대여소명                     |
|----+----------+----------+----------------+--------------+-----------------------------------|
|  0 | 20200101 |        0 |           8313 |         2358 | 구룡초사거리 (현대아파트10동 앞 ) |
|  1 | 20200101 |        0 |          25512 |         2325 | 대치삼성아파트 앞 교차로          |
|  2 | 20200101 |        0 |          46405 |         2343 | 세곡사거리 대왕파출소 앞          |
+----+----------+----------+----------------+--------------+-----------------------------------+


In [17]:
print(tabulate(final_df.head(3), headers='keys', tablefmt='psql', showindex=True))

+----+--------------+------------------------+------------+----------+--------+----------------+---------+
|    |   대여소번호 | 대여 대여소명          | 날짜       |   시간대 | 날씨   |   평균기온(°C) |   Pm2.5 |
|----+--------------+------------------------+------------+----------+--------+----------------+---------|
|  0 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        0 | 비옴   |           -2.2 |      24 |
|  1 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        1 | 비옴   |           -2.2 |      21 |
|  2 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        2 | 비옴   |           -2.2 |      15 |
+----+--------------+------------------------+------------+----------+--------+----------------+---------+


In [18]:
# 날짜 형식 변경
df_people['날짜'] = pd.to_datetime(df_people['날짜'], format='%Y%m%d')

# 날짜를 원하는 형식으로 다시 문자열로 변환
df_people['날짜'] = df_people['날짜'].dt.strftime('%Y-%m-%d')

# df 데이터프레임의 '대여 대여소명' 열의 앞 뒤 공백 제거
final_df['대여 대여소명'] = final_df['대여 대여소명'].str.strip()
df_people['대여 대여소명'] = df_rental['대여 대여소명'].str.strip()

# df_people 칼럼의 측정시간 값과 일치하는 것을 기준으로 두 데이터프레임을 날짜를 기준으로 병합
final_df = pd.merge(final_df, df_people, left_on=['날짜', '시간대','대여소번호'], right_on=['날짜', '시간대','대여소번호'], how='left')

print(final_df.shape)
print(tabulate(final_df.head(), headers='keys', tablefmt='psql', showindex=True))

(836173, 9)
+----+--------------+------------------------+------------+----------+--------+----------------+---------+----------------+------------------------+
|    |   대여소번호 | 대여 대여소명_x        | 날짜       |   시간대 | 날씨   |   평균기온(°C) |   Pm2.5 |   유동인구(명) | 대여 대여소명_y        |
|----+--------------+------------------------+------------+----------+--------+----------------+---------+----------------+------------------------|
|  0 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        0 | 비옴   |           -2.2 |      24 |          27965 | 3호선 매봉역 3번출구앞 |
|  1 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        1 | 비옴   |           -2.2 |      21 |          28337 | 3호선 매봉역 3번출구앞 |
|  2 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        2 | 비옴   |           -2.2 |      15 |          28410 | 3호선 매봉역 3번출구앞 |
|  3 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        3 | 비옴   |           -2.2 |      12 |          28422 | 3호선 매봉역 3번출구앞 |
|  4 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        

In [19]:
# 제외한 열만 선택하여 새로운 데이터프레임 생성
final_df = final_df.drop(columns='대여 대여소명_y')

# '보관소(대여소)명' 열의 이름을 '대여 대여소명'로 변경
final_df = final_df.rename(columns={'대여 대여소명_x': '대여 대여소명'})


In [20]:
print(tabulate(final_df.head(10), headers='keys', tablefmt='psql', showindex=True))

+----+--------------+------------------------+------------+----------+--------+----------------+---------+----------------+
|    |   대여소번호 | 대여 대여소명          | 날짜       |   시간대 | 날씨   |   평균기온(°C) |   Pm2.5 |   유동인구(명) |
|----+--------------+------------------------+------------+----------+--------+----------------+---------+----------------|
|  0 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        0 | 비옴   |           -2.2 |      24 |          27965 |
|  1 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        1 | 비옴   |           -2.2 |      21 |          28337 |
|  2 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        2 | 비옴   |           -2.2 |      15 |          28410 |
|  3 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        3 | 비옴   |           -2.2 |      12 |          28422 |
|  4 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        4 | 비옴   |           -2.2 |      15 |          28278 |
|  5 |         2335 | 3호선 매봉역 3번출구앞 | 2020-01-01 |        5 | 비옴   |           -2.2 |     

In [21]:
final_df.to_csv('/content/drive/MyDrive/Acorn/Project/df_final2020.csv', index=False)

## 2020 데이터 최종 Merge

In [ ]:
import pandas as pd
from tabulate import tabulate

In [ ]:
df1=pd.read_csv(r'/content/drive/MyDrive/Acorn/Project/2020data/20년5분데이터.csv')
print(tabulate(df1.head(10), headers='keys', tablefmt='psql', showindex=True))

In [ ]:
final_df=pd.read_csv(r'/content/drive/MyDrive/Acorn/Project/df_final2020.csv')
print(tabulate(final_df.head(10), headers='keys', tablefmt='psql', showindex=True))
print(len(final_df[final_df['대여 대여소명']=='3호선 매봉역 3번출구앞']))

In [ ]:
# 날짜 형식 변경
df1['기준_날짜'] = pd.to_datetime(df1['기준_날짜'], format='%Y%m%d')

# 날짜를 원하는 형식으로 다시 문자열로 변환
df1['기준_날짜'] = df1['기준_날짜'].dt.strftime('%Y-%m-%d')

# '기준_날짜', '시간대', '대여소명'을 기준으로 두 데이터프레임을 병합-  대여건수랑 반납건수가 둘다 없는 것은 삭제하기 위해서 inner join 사용 -> 시간대는 있지만, 그에 따른 대여건수랑 반납건수가 없기에 inner join으로 그 시간대에 있는 정보만 가져옴
merged_df = pd.merge(final_df,df1, left_on=['날짜', '시간대', '대여 대여소명'], right_on=['기준_날짜', '시간대', '대여소명'],  how='inner')

# 결과 출력
print(tabulate(merged_df.head(10), headers='keys', tablefmt='psql', showindex=True))

In [ ]:
# 주어진 데이터프레임에서 대여소번호, 대여소명, 기준_날짜 열을 제외하고 새로운 데이터프레임 생성
final_df = merged_df.drop(['대여소번호', '대여소명', '기준_날짜'], axis=1)

# 대여소 ID 칼럼을 맨 앞으로 이동
cols = list(final_df.columns)
cols = ['대여소ID'] + [col for col in cols if col != '대여소ID']
final_df = final_df[cols]

#대여 대여소 칼럼명 변경
final_df = final_df.rename(columns={'대여 대여소명': '대여소명'})

In [ ]:
print(final_df.shape)

print(final_df[final_df['대여소명']=='3호선 매봉역 3번출구앞'])
print(tabulate(final_df.head(100), headers='keys', tablefmt='psql', showindex=True))

In [ ]:
final_df.to_csv('/content/drive/MyDrive/Acorn/Project/df_final_final2020_.csv', index=False)